In [ ]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import brown

In [ ]:
#download the treebank corpus from nltk
nltk.download('treebank')
#download the universal tagset from nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[0:1])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]]


In [ ]:
pos=dict()
uni_words=[]
for i in range(len(nltk_data[0:5])):
  nltk_data[i].insert(0,('<s>','START'))
  nltk_data[i].insert(len(nltk_data[i])-1,('</s>','END'))
  # print(nltk_data[i])
  for j in range(len(nltk_data[i])):
    if nltk_data[i][j][0] not in uni_words:
      uni_words.append(nltk_data[i][j][0])
    if nltk_data[i][j][1] in pos:
      tempdict=pos[nltk_data[i][j][1]]
      # print(nltk_data[i][j][1])
      if nltk_data[i][j][0] in tempdict:
        count = tempdict[nltk_data[i][j][0]]
        tempdict.update({nltk_data[i][j][0]:count+1})
        # print(nltk_data[i][j][1])
        pos[nltk_data[i][j][1]].update(tempdict)
      else:
        tempdict.update({nltk_data[i][j][0]:1})
        pos[nltk_data[i][j][1]].update(tempdict)
    else:
      tempdict=dict({nltk_data[i][j][0]:1})
      pos.update({nltk_data[i][j][1]:tempdict})
    
pos

{'START': {'<s>': 5},
 'NOUN': {'Pierre': 1,
  'Vinken': 2,
  'years': 3,
  'board': 1,
  'director': 2,
  'Nov.': 1,
  'Mr.': 1,
  'chairman': 2,
  'Elsevier': 1,
  'N.V.': 1,
  'Dutch': 1,
  'group': 2,
  'Rudolph': 1,
  'Agnew': 1,
  'Consolidated': 1,
  'Gold': 1,
  'Fields': 1,
  'PLC': 1,
  'conglomerate': 1,
  'form': 1,
  'asbestos': 2,
  'Kent': 1,
  'cigarette': 1,
  'filters': 1,
  'percentage': 1,
  'cancer': 1,
  'deaths': 1,
  'workers': 1,
  'researchers': 2,
  'fiber': 1,
  'crocidolite': 1,
  'lungs': 1,
  'exposures': 1,
  'symptoms': 1,
  'decades': 1},
 '.': {',': 10, '.': 5},
 'NUM': {'61': 1, '29': 1, '55': 1, '30': 1},
 'ADJ': {'old': 2,
  'nonexecutive': 2,
  'former': 1,
  'British': 1,
  'industrial': 1,
  'high': 1,
  'resilient': 1,
  'brief': 1,
  'later': 1},
 'VERB': {'will': 1,
  'join': 1,
  'is': 2,
  'publishing': 1,
  'was': 1,
  'named': 1,
  'used': 1,
  'make': 1,
  'has': 1,
  'caused': 1,
  'exposed': 1,
  'reported': 1,
  'enters': 1,
  'causin

In [ ]:
pos_count=dict()
for i in range(len(nltk_data[0:5])):
  for j in range(len(nltk_data[i])):
    if nltk_data[i][j][1] not in pos_count:
      pos_count.update({nltk_data[i][j][1]:1})
    else:
      count = pos_count[nltk_data[i][j][1]]
      pos_count.update({nltk_data[i][j][1]:count+1})
pos_count

{'START': 5,
 'NOUN': 43,
 '.': 15,
 'NUM': 4,
 'ADJ': 11,
 'VERB': 17,
 'DET': 11,
 'ADP': 12,
 'END': 5,
 'CONJ': 1,
 'X': 9,
 'ADV': 4,
 'PRT': 4,
 'PRON': 3}

In [ ]:
pos_bi=dict()
for i in range(len(nltk_data[0:5])):
  for j in range(len(nltk_data[i])-1):
    tup = (nltk_data[i][j][1],nltk_data[i][j+1][1])
    if tup in pos_bi:
      count = pos_bi[tup]
      pos_bi.update({tup:count+1})
    else:
      pos_bi.update({tup:1})
pos_bi_li = list(pos_bi.items())

###Transition Matrix

In [ ]:
trans_mat = np.zeros((len(pos_count.keys()),len(pos_count.keys())))
transition_matrix = pd.DataFrame(trans_mat,index=pos_count.keys(),columns=pos_count.keys())
for i in range(len(pos_bi_li)):
  r=pos_bi_li[i][0][0]
  c=pos_bi_li[i][0][1]
  transition_matrix[c][r]=pos_bi_li[i][1]/pos_count[pos_bi_li[i][0][0]]
print("Transition Matrix")
transition_matrix

Transition Matrix


,START,NOUN,.,NUM,ADJ,VERB,DET,ADP,END,CONJ,X,ADV,PRT,PRON
START,0.0,0.600000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NOUN,0.0,0.279070,0.162791,0.023256,0.069767,0.139535,0.023256,0.209302,0.046512,0.000000,0.000000,0.023256,0.023256,0.000000
.,0.0,0.200000,0.000000,0.133333,0.000000,0.200000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NUM,0.0,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
ADJ,0.0,0.545455,0.181818,0.000000,0.090909,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000
VERB,0.0,0.235294,0.000000,0.000000,0.000000,0.176471,0.176471,0.000000,0.000000,0.000000,0.294118,0.058824,0.058824,0.000000
DET,0.0,0.545455,0.000000,0.000000,0.363636,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000
ADP,0.0,0.416667,0.083333,0.083333,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333
END,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CONJ,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###Emission Matrix

In [ ]:
pos_li = list(pos.items())
emm_mat = np.zeros((len(uni_words),len(pos_count.keys())))
emission_matrix = pd.DataFrame(emm_mat,index=uni_words,columns=pos_count.keys())
for i in range(len(pos_li)):
  c=pos_li[i][0]
  tempkeydict=list(pos_li[i][1].keys())
  for j in range(len(tempkeydict)):
    emission_matrix[c][tempkeydict[j]]=pos_li[i][1][tempkeydict[j]]/pos_count[c]
print("Emission Matrix")
emission_matrix

Emission Matrix


,START,NOUN,.,NUM,ADJ,VERB,DET,ADP,END,CONJ,X,ADV,PRT,PRON
<s>,1.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
Pierre,0.0,0.023256,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
Vinken,0.0,0.046512,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
",",0.0,0.000000,0.666667,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
61,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0
decades,0.0,0.023256,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
later,0.0,0.000000,0.000000,0.00,0.090909,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0
said,0.0,0.000000,0.000000,0.00,0.000000,0.058824,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0


##Viterbi

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
random.seed(1234)      
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
print(test_run)
test_run_base = [tup for sent in test_run for tup in sent]
print(test_run_base)
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_tagged_words)

[[('The', 'DET'), ('company', 'NOUN'), ('is', 'VERB'), ('contesting', 'VERB'), ('the', 'DET'), ('fine', 'NOUN'), ('.', '.')], [('``', '.'), ('I', 'PRON'), ("'m", 'VERB'), ('starting', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('see', 'VERB'), ('more', 'ADJ'), ('business', 'NOUN'), ('transactions', 'NOUN'), (',', '.'), ("''", '.'), ('says', 'VERB'), ('*T*-2', 'X'), ('Andrea', 'NOUN'), ('West', 'NOUN'), ('of', 'ADP'), ('American', 'NOUN'), ('Telephone', 'NOUN'), ('&', 'CONJ'), ('Telegraph', 'NOUN'), ('Co.', 'NOUN'), (',', '.'), ('*-3', 'X'), ('noting', 'VERB'), ('growing', 'VERB'), ('interest', 'NOUN'), ('in', 'ADP'), ('use', 'NOUN'), ('of', 'ADP'), ('900', 'NUM'), ('service', 'NOUN'), ('for', 'ADP'), ('stock', 'NOUN'), ('sales', 'NOUN'), (',', '.'), ('software', 'NOUN'), ('tutorials', 'NOUN'), ('and', 'CONJ'), ('even', 'ADV'), ('service', 'NOUN'), ('contracts', 'NOUN'), ('.', '.')], [('He', 'PRON'), ('added', 'VERB'), (',', '.'), ('``', '.'), ('This', 'DET'), ('has', 'VERB'), ('nothing', 'N

In [ ]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
print(tagged_seq)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
print(check)
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

[('The', 'DET'), ('company', 'NOUN'), ('is', 'VERB'), ('contesting', 'NUM'), ('the', 'DET'), ('fine', 'NOUN'), ('.', '.'), ('``', '.'), ('I', 'PRON'), ("'m", 'VERB'), ('starting', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('see', 'VERB'), ('more', 'ADV'), ('business', 'NOUN'), ('transactions', 'NOUN'), (',', '.'), ("''", '.'), ('says', 'VERB'), ('*T*-2', 'X'), ('Andrea', 'NUM'), ('West', 'NOUN'), ('of', 'ADP'), ('American', 'ADJ'), ('Telephone', 'NOUN'), ('&', 'CONJ'), ('Telegraph', 'NOUN'), ('Co.', 'NOUN'), (',', '.'), ('*-3', 'X'), ('noting', 'VERB'), ('growing', 'VERB'), ('interest', 'NOUN'), ('in', 'ADP'), ('use', 'NOUN'), ('of', 'ADP'), ('900', 'NUM'), ('service', 'NOUN'), ('for', 'ADP'), ('stock', 'NOUN'), ('sales', 'NOUN'), (',', '.'), ('software', 'NOUN'), ('tutorials', 'NUM'), ('and', 'CONJ'), ('even', 'ADV'), ('service', 'NOUN'), ('contracts', 'NOUN'), ('.', '.'), ('He', 'PRON'), ('added', 'VERB'), (',', '.'), ('``', '.'), ('This', 'DET'), ('has', 'VERB'), ('nothing', 'NOUN'), ('